<a href="https://colab.research.google.com/github/nak650228/GFPGAN_video_restoration/blob/main/GFPGAN_inference_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPFGANを用いた動画修復


# 1. 初期設定

In [1]:
#@title ##**割り当てられたGPUの確認**
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.
#@markdown *割り当てられたGPUのメモリ容量が小さいと、修復が失敗する場合があります。できる限りTesla V100,P100,T4以上を利用してください。*

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 460.32.03, 16280 MiB


In [2]:
#@title　##**基本ライブラリ等のインストール**
!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python

from google.colab import files
import shutil
import youtube_dl
import ffmpeg
import cv2
import glob
import os
import moviepy.editor as mpy
from IPython.display import clear_output
clear_output()

In [3]:
#@title ##**GFPGANをGithubからクローン**

# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
clear_output()

# 2. 動画ファイルの指定と調整

In [ ]:
#@title ##**ビデオのダウンロード** { display-mode: "form" }
#@markdown *ビデオへのリンク（YouTubeやTwitterなど）を入力するか、source_urlフィールドを空白にしてください（空白にした場合、コンピューターからビデオをアップロードするよう求められます）。*

source_url = '' #@param {type:"string"}

%cd /content
! rm -f /content/*.mp4

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4

clear_output()

fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))
print ("FPS of VIDEO: ",fps_of_video)
print ("Frames of VIDEO: ",frames_of_video)
print ("Width of VIDEO: ",width_of_video)
print ("Height of VIDEO: ",height_of_video)


#@markdown *ダウンロードは失敗することがあります。もし失敗しても再度実行することでダウンロードできることがあります。*

#@markdown *ダウンロードに成功すると、1秒間当たりのフレーム数や、全体のフレーム数、縦横の解像度が出力結果に表示されます（下の例を参照）この結果を参考にして動画の調整を行ってください。*

#@markdown *FPS of VIDEO:  29*

#@markdown *Frames of VIDEO:  448*

#@markdown *Width of VIDEO:  640*

#@markdown *Height of VIDEO:  480*

In [5]:
#@title ##**ダウンロードした動画を表示** { display-mode: "form" }
#@markdown *what_nextにplayを指定すると、Youtubeからダウンロードした動画を表示します。解像度が大きな動画は表示が失敗することがあります。その場合はwhat_nextにdownloadを指定して、PCなどにダウンロードして確認して下さい*

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/downloaded_video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/downloaded_video.mp4')

In [ ]:
#@markdown *1分以上の動画をダウンロードすることはお勧めできません。また、タイトルに「スペース」や「ドット」が含まれている動画はアップロードしないでください。*

#@markdown *実行中にエラーが発生した場合は、このブロックを再度実行します。*

#@title ##**動画の調整** { display-mode: "form" }
#@markdown *動画の長さを変更することができます。*

#@markdown **動画の長さを変更する場合は、その開始時間と終了時間を指定して下さい。**
target_start = '00:00:00' #@param {type:"string"}
target_end = '00:00:00' #@param {type:"string"}

#@markdown **動画の解像度を変更する場合は、新しい解像度を指定してください（例 640 x 480）。この値を指定しない場合は、元の解像度がそのまま使われます。**
#@markdown **GFP GANの出力結果は解像度が2倍になるため、この値を大きくすることはお勧めできません**
#@markdown **できる限り縦の解像度は540以下にすることをお勧めします。**
width = "" #@param {type:"string"}
height = "" #@param {type:"string"}

rescale = ""
if width != '' and height != '':
#  rescale = f"-s {width}x{height}"
  rescale = f"-vf scale={width}:{height}"

if os.path.isfile("/content/video.mp4") :
    !rm -f /content/video.mp4
if os.path.isfile("/content/cropped_video.mp4") :
    !rm -f /content/cropped_video.mp4

if os.path.isfile("/content/cropped_video.mp4") :
    !rm -f cropped_video.mp4
!ffmpeg -i /content/downloaded_video.mp4 $rescale -ss $target_start -to $target_end /content/cropped_video.mp4
#clear_output()

!cp /content/cropped_video.mp4 /content/video.mp4


clear_output()
file_name="/content/video.mp4"
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))
print ("FPS of VIDEO: ",fps_of_video)
print ("Frames of VIDEO: ",frames_of_video)
print ("Width of VIDEO: ",width_of_video)
print ("Height of VIDEO: ",height_of_video)

In [ ]:
#@title ##**サイズ調整後の動画を表示（修復対象）** { display-mode: "form" }
#@markdown *what_nextにplayを指定すると、動画の調整で指定した結果を表示します。解像度が大きな動画は表示が失敗することがあります。その場合はwhat_nextにdownloadを指定して、PCなどにダウンロードして確認して下さい*
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/video.mp4')

# 3. GFPGAN メイン処理

In [8]:
#@title ##**ビデオを静止画フレームに分解する** { display-mode: "form" }
upload_folder = "/content/GFPGAN/inputs/upload"
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)

os.mkdir(upload_folder)

%cd /content/GFPGAN/inputs/upload

!ffmpeg -i /content/video.mp4 %09d.png

clear_output()

In [ ]:
#@title ##**GFPGANによる修復** { display-mode: "form" }
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
%cd /content/GFPGAN
!rm -rf results
!python inference_gfpgan.py --upscale 2 --test_path inputs/upload --save_root results --model_path experiments/pretrained_models/GFPGANCleanv1-NoCE-C2.pth --bg_upsampler realesrgan
clear_output()

!ls results/cmp

In [ ]:
#@title ##**修復した静止画を動画に復元** { display-mode: "form" }
#!ffmpeg -vsync 0 -hwaccel cuvid -c:v mjpeg_cuvid -framerate 30 -i /content/TecoGAN/results/My_video/*.png -c:v h264_nvenc quaid2.mp4
if os.path.isfile("/content/restored_video.mp4") :
    !rm -f /content/restored_video.mp4

!ffmpeg -f image2 -framerate 30 -i /content/GFPGAN/results/restored_imgs/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/restored_video.mp4
!rm -f /content/video.mp4
!cp /content/restored_video.mp4 /content/video.mp4

clear_output()

In [ ]:
#@title ##**修復した動画を表示** { display-mode: "form" }
#@markdown *what_nextにplayを指定すると、GFPGANで修復した結果を表示します。解像度が大きな動画は表示が失敗することがあります。その場合はwhat_nextにdownloadを指定して、PCなどにダウンロードして確認して下さい*
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/video.mp4')